# Recommender Systems

### Libraries

In [ ]:
import json
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, accuracy, NormalPredictor, KNNBasic, SVD
from sklearn.metrics.pairwise import cosine_similarity
from surprise.model_selection import train_test_split
from collections import Counter, defaultdict
from utils import load_filtered_data

### Reviews data

In [ ]:
# major variables

city = 'Springfield'
city_data = load_filtered_data(city)

In [ ]:
# data frame of business reviews by users

reviews = city_data['review'][['review_id', 'business_id', 'user_id', 'stars']]
reviews = reviews.groupby(['user_id', 'business_id'])['stars'].mean().reset_index()
reviews.columns = ['user_id', 'business_id', 'rating']

In [ ]:
reviews.sample(5)

In [ ]:
# number of ratings

len(reviews['rating'])

In [ ]:
# number of users who have reviewed

len(reviews['user_id'].unique())

In [ ]:
# ratings statistics

reviews['rating'].describe()

In [ ]:
# ratings distribution

reviews['rating'].hist()

### User-business matrix

In [ ]:
# city matrix

city_matrix = reviews.pivot(index='user_id', columns='business_id', values='rating')

In [ ]:
city_matrix.sample(5)

In [ ]:
# city matrix sparsity

print(f"{city_matrix.notnull().sum().sum() / (city_matrix.shape[0] * city_matrix.shape[1]):.2%}")

In [ ]:
# distribution of total number of items per user

businesses_per_user = city_matrix.notnull().sum(axis=1)
plt.figure(figsize=(10, 6))
plt.hist(businesses_per_user, bins=50, color='skyblue', edgecolor='black')
plt.title('Distribution of Total Number of Businesses per User')
plt.xlabel('Number of Businesses per User')
plt.ylabel('Number of Users')
plt.grid(True)
plt.show()

In [ ]:
# distribution of total number of users per business

users_per_business = city_matrix.notnull().sum()
plt.figure(figsize=(10, 6))
plt.hist(users_per_business, bins=50, color='lightgreen', edgecolor='black')
plt.title('Distribution of Total Number of Users per Business')
plt.xlabel('Number of Users per Business')
plt.ylabel('Number of Businesses')
plt.grid(True)
plt.show()

In [ ]:
# distribution of mean ratings per user

mean_ratings_per_user = city_matrix.mean(axis=1)
plt.figure(figsize=(10, 6))
plt.hist(mean_ratings_per_user, bins=50, color='orange', edgecolor='black')
plt.title('Distribution of Mean Ratings per User')
plt.xlabel('Mean Ratings')
plt.ylabel('Number of Users')
plt.grid(True)
plt.show()

### Community matrices

In [ ]:
# user-business matrix for each community

connection = 'priority_combined'
with open(f'communities/{city}_{connection}_communities.json') as f:
    data = json.load(f)

community_matrices = {}
for i, community in enumerate(data['communities']):
    community_matrices[i] = reviews[reviews['user_id'].isin(community)]
    community_matrices[i] = community_matrices[i].pivot_table(index='user_id', columns='business_id', values='rating')

In [ ]:
community_matrices[0].sample(5)

In [ ]:
# data sparsity

sparsity = {}

for community_id, matrix in community_matrices.items():
    sparsity[community_id] = matrix.notnull().sum().sum() / (matrix.shape[0] * matrix.shape[1])

for community_id, sparsity_value in sparsity.items():
    print(f"Community {community_id}: {sparsity_value:.2%} sparsity")

# Plot histogram
plt.figure(figsize=(8, 6))
plt.hist(sparsity.values(), bins=20, color='skyblue', edgecolor='black')
plt.title('Distribution of Sparsity per Community')
plt.xlabel('Sparsity')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

### Train and test sets

In [ ]:
community_trainsets = {}
community_testsets = {}

for community_id, matrix in community_matrices.items():
    matrix_filtered = matrix[matrix >= 3].dropna(axis=0, how='all')

    user_review_counts = matrix_filtered.apply(lambda row: row.count(), axis=1)
    users_with_min_reviews = user_review_counts[user_review_counts >= 3].index.tolist()
    print(user_review_counts)
    matrix_filtered = matrix_filtered.loc[users_with_min_reviews]
    df = matrix_filtered.stack().reset_index()
    df.columns = ['user_id', 'business_id', 'rating']

    trainset = defaultdict(list)
    testset = defaultdict(list)
    for user_id, group in df.groupby('user_id'):
        num_reviews = len(group)
        train_size = int(num_reviews * 0.8)
        train_reviews = group[:train_size]
        test_reviews = group[train_size:]
        trainset[community_id].extend(train_reviews.values.tolist())
        testset[community_id].extend(test_reviews.values.tolist())

    reader = Reader(rating_scale=(1, 5))
    train_data = Dataset.load_from_df(pd.DataFrame(trainset[community_id], columns=['user_id', 'business_id', 'rating']), reader)
    test_data = Dataset.load_from_df(pd.DataFrame(testset[community_id], columns=['user_id', 'business_id', 'rating']), reader)

    community_trainsets[community_id] = train_data.build_full_trainset()
    community_testsets[community_id] = test_data.build_full_trainset().build_testset()

In [ ]:
# check a community trainset

trainset = community_trainsets[0]
for user_id, item_id, rating in trainset.all_ratings():
    print("User:", user_id, "Item:", item_id, "Rating:", rating)

In [ ]:
# check a community testset

testset = community_testsets[0]
for user_id, item_id, rating in testset:
    print("User:", user_id, "Item:", item_id, "Rating:", rating)

In [ ]:
# trainsets statistics

for community_id, trainset in community_trainsets.items():
    print(f"Community {community_id}: {len(trainset.all_users())} users")

    businesses_per_user = [len(trainset.ur[user_id]) for user_id in trainset.all_users()]
    print(f"Average number of businesses per user: {np.mean(businesses_per_user):.2f}")

    users_per_business = [len(trainset.ir[business_id]) for business_id in trainset.all_items()]
    print(f"Average number of users per business: {np.mean(users_per_business):.2f}")

    mean_ratings_per_user = [np.mean([ratings for (_, ratings) in trainset.ur[user_id]]) for user_id in trainset.all_users()]
    print(f"Average mean ratings per user: {np.mean(mean_ratings_per_user):.2f}")

    ratings_distribution = [ratings for (_, _, ratings) in trainset.all_ratings()]
    print(f"Average ratings: {np.mean(ratings_distribution):.2f}")

    print("\n")

In [ ]:
# 5 most popular businesses in each community

for community_id, trainset in community_trainsets.items():
    business_ratings_count = Counter([len(trainset.ir[business_id]) for business_id in trainset.all_items()])
    most_popular_businesses = business_ratings_count.most_common(5)


    print(f"\nMost popular businesses in Community {community_id}:")
    for business_id, count in most_popular_businesses:
        print(f"Business ID: {business_id}, Number of Ratings: {count}")

### Popularity

In [ ]:
def popular_recommendations(trainset, top_n=10):
    '''
    Returns the top n most popular businesses in the trainset (number of reviews and average rating)
    '''
    business_counts = defaultdict(int)
    business_ratings = defaultdict(float)

    for _, business_id, _ in trainset.all_ratings():
        business_counts[business_id] += 1
        business_ratings[business_id] += rating

    popular_businesses = []

    for business_id, count in business_counts.items():
        avg_rating = business_ratings[business_id] / count if count > 0 else 0
        popularity_score = (count*0.2 + avg_rating*0.8) / 2
        popular_businesses.append((business_id, popularity_score))

    popular_businesses.sort(key=lambda x: x[1], reverse=True)
    top_n = popular_businesses[:top_n]
    return [trainset.to_raw_iid(i) for i, _ in top_n]

In [ ]:
# most popular businesses in each community

for community_id, trainset in community_trainsets.items():
    popular_businesses = popular_recommendations(trainset, 5)
    print(f"Community {community_id}: {popular_businesses}")

### Modelling

In [ ]:
def evaluate_algorithm(algo, trainset, testset):
    '''
    Evaluate the algorithm using RMSE
    '''
    if (len(trainset.all_users()) == 0) or (len(trainset.all_items()) == 0):
        print('No data.')
        return None
    
    algo.fit(trainset)
    predictions = algo.test(testset)

    print(predictions)
    rmse = accuracy.rmse(predictions)
    return rmse

In [ ]:
# random recommender for each community

random_algo = NormalPredictor()
for community_id, trainset in community_trainsets.items():
    testset = community_testsets[community_id]
    random_rmse = evaluate_algorithm(random_algo, trainset, testset)

In [ ]:
# user-based collaborative filtering for each community

ubcf_algo = KNNBasic(sim_options={'user_based': True})
user_recommendations = {}

for community_id, trainset in community_trainsets.items():
    testset = community_testsets[community_id]
    #ubcf_rmse = evaluate_algorithm(ubcf_algo, trainset, testset)
    ubcf_algo.fit(trainset)
    predictions = ubcf_algo.test(testset)
    for uid, iid, true_r, est, _ in predictions:
        if uid not in user_recommendations:
            user_recommendations[uid] = []
        user_recommendations[uid].append((iid, est))
    
    

for user_id, recommendations in user_recommendations.items():
    print("User:", user_id)
    for business_id, rating in recommendations:
        print("Business:", business_id, "Estimated Rating:", rating)


In [ ]:
# item-based collaborative filtering for each community

ibcf_algo = KNNBasic(sim_options={'user_based': False})

for community_id, trainset in community_trainsets.items():
    testset = community_testsets[community_id]
    ibcf_rmse = evaluate_algorithm(ibcf_algo, trainset, testset)


In [ ]:
# Singular Value Decomposition (SVD) for each community
svd_recommendations = {}

svd_algo = SVD()
for community_id, trainset in community_trainsets.items():
    testset = community_testsets[community_id]
    #svd_rmse = evaluate_algorithm(svd_algo, trainset, testset)

    svd_algo.fit(trainset)
    predictions = svd_algo.test(testset)
    for uid, iid, true_r, est, _ in predictions:
        if uid not in svd_recommendations:
            svd_recommendations[uid] = []
        svd_recommendations[uid].append((iid, est))

### Top recommendations

In [ ]:
def recommend_top_n(algo, trainset, user_id, n=10):
    '''
    Recommend top n items for a user using a recommender model
    '''
    user_ratings = trainset.ur[user_id]
    items = [item_id for (item_id, _) in user_ratings]
    item_scores = {}
    # this is actually not the most correct way to do this, but it works
    for item_id in trainset.all_items():
        if item_id not in items:
            prediction = algo.predict(trainset.to_raw_uid(user_id), trainset.to_raw_iid(item_id), verbose=True)
            item_scores[item_id] = prediction.est
    
    top_items = sorted(item_scores, key=item_scores.get, reverse=True)[:n]
    
    return [trainset.to_raw_iid(i) for i in top_items]

In [ ]:
# major variables

community_id = 0
n = 5
pos_rating = 4

In [ ]:
# recommendations for each user in the community

# for user_id, recommendations in community_recommendations.items():
#     print(f"User {user_id}: {recommendations}")

### Evaluation

In [ ]:
# convert testsets to dataframes

community_test_dfs = {}
for community_id, testset in community_testsets.items():
    community_test_dfs[community_id] = pd.DataFrame(testset, columns=['user_id', 'item_id', 'rating'])

In [ ]:
# generate top recommendations for each user in a community

community_recommendations = {}
for index in range(len(community_trainsets[community_id].all_users())):
    user_id = community_trainsets[community_id].all_users()[index]
    recommendations = recommend_top_n(svd_algo, community_trainsets[0], user_id, n)
    community_recommendations[user_id] = recommendations


In [ ]:
community_test_dfs[0].sample(5)

In [ ]:
# distribution of number of ratings per user in each community

for community_id, test_df in community_test_dfs.items():
    user_rating_counts = test_df.groupby("user_id")['rating'].apply(len)
    rating_count_distribution = user_rating_counts.value_counts()
    print(f"Community {community_id} - Distribution of Number of Ratings per User:")
    print(rating_count_distribution)

In [ ]:
# number of users in the test set that exist in the train set

for community_id, test_df in community_test_dfs.items():
    df_testset_pos = test_df[test_df["rating"] > pos_rating]
    users = []
    for u in df_testset_pos["user_id"].unique():
        try:
            community_trainsets[community_id].to_inner_uid(u)
            users.append(u)
        except ValueError:
            continue
    
    print(f"Community {community_id}: Number of users in the test set that exist in the train set:", len(users))

In [ ]:
# evaluate recommendations for each user in a community

print(f"Community {community_id} - Recommendations Evaluation:")
test_df = community_test_dfs[community_id]

for user_id, recommendations in community_recommendations.items():
    print("user_id:", user_id)
    gt = test_df[(test_df['user_id']==user_id) & (test_df['rating']>pos_rating)].item_id.to_list()
    print("ground truth:", gt)
    print("recommendations:", recommendations)
    print(f"hits: {len(set(gt).intersection(set(recommendations)))} / {n}")

## Content-based

In [ ]:
merged = pd.merge(city_data['review'], city_data['user'], on='user_id', how='left')
merged = pd.merge(merged, city_data['business'], on='business_id', how='left')
merged = merged.drop_duplicates(subset=['user_id', 'business_id'])

merged.rename(columns={'stars_x': 'rating'}, inplace=True)
merged.rename(columns={'stars_y': 'stars'}, inplace=True)

merged = merged[['user_id', 'business_id', 'rating', 'categories', 'stars']]

#### creates a column for each category

In [ ]:
all_categories = set()

for categories_list in merged['categories']:
    x = categories_list.split(',')
    for i in x:
        all_categories.add(i.strip())

category_columns = list(all_categories)

category_df = pd.DataFrame(0, index=merged.index, columns=category_columns)

for i, row in merged.iterrows():
    for category in row['categories'].split(','):
        category_df.at[i, category.strip()] = 1

category_counts = category_df.sum()
top_categories = category_counts.nlargest(200).index.to_list()

merged = pd.concat([merged, category_df], axis=1)

merged.drop('categories', axis=1, inplace=True)

merged = merged.loc[:, (merged != 0).any(axis=0)]


#### New train sets

In [ ]:
community_trainsets_content = {}
community_testsets_content = {}

communities = {}

for idx, community_users in enumerate(data['communities']):
    communities[idx] = community_users

for community_id, community in communities.items():    
    community_reviews = merged[merged['user_id'].isin(community)]
    community_reviews = community_reviews[community_reviews['rating'] >= 3]
    user_review_counts = community_reviews.groupby('user_id').size()
    user_review_counts = user_review_counts.reset_index()
    user_review_counts.columns = ['user_id', 'review_count']

    users_with_min_reviews = user_review_counts[user_review_counts['review_count'] >= 3]['user_id'].tolist()

    merged_filtered = community_reviews[community_reviews['user_id'].isin(users_with_min_reviews)]

    trainset = []
    testset = []
    for user_id, group in merged_filtered.groupby('user_id'):
        num_reviews = len(group)
        train_size = int(num_reviews * 0.7)
        train_reviews = group[:train_size] 
        test_reviews = group[train_size:]  
        trainset.append(train_reviews)
        testset.append(test_reviews)

    if trainset: 
        trainset_df = pd.concat(trainset)
        community_trainsets_content[community_id] = trainset_df

    if testset:  
        testset_df = pd.concat(testset)
        community_testsets_content[community_id] = testset_df



In [ ]:
def content_based_recommendations( trainset, top_n=3):
    
    similarity_matrix = cosine_similarity(trainset[["stars","Watches","Kids Hair Salons","Soup","Emergency Pet Hospital","Bars","Pressure Washers","Car Window Tinting","Tacos","Chinese","Occupational Therapy","Knitting Supplies","Disc Golf","Coffee & Tea","Donuts","Physical Therapy","Hair Stylists","Boot Camps","Desserts","Eyelash Service","Computers","Italian","Restaurants","Senior Centers","Mags","Caterers","Financial Advising","Performing Arts","Dry Cleaning","Funeral Services & Cemeteries","Bagels","Car Dealers","Trampoline Parks","Cosmetic Surgeons","Contractors","Flowers & Gifts","Sandwiches","Amusement Parks","Southern","Country Clubs","Cheesesteaks","Shaved Ice","Burgers","Skin Care","Tires","Gold Buyers","Recording & Rehearsal Studios","Event Photography","Weight Loss Centers","Vietnamese","Nail Salons","Event Planning & Services","Psychologists","Hair Salons","Auto Detailing","Indoor Playcentre","Specialty Schools","Golf","Body Shops","Comfort Food","Arcades","Blow Dry/Out Services","Employment Agencies","Heating & Air Conditioning/HVAC","Fitness & Instruction","Hookah Bars","Music & Video","Office Equipment","Junk Removal & Hauling","Building Supplies","Tobacco Shops","Family Practice","Mobile Phone Accessories","Plumbing","Photographers","Men's Clothing","Barbers","Hardware Stores","Middle Eastern","Food Court","Massage Therapy","Pet Services","Health Retreats","Home Window Tinting","Permanent Makeup","Pubs","Shoe Stores","Shades & Blinds","Electricians","General Dentistry","Real Estate Services","Windshield Installation & Repair","Fabric Stores","Diagnostic Services","Pest Control","Discount Store","Videos & Video Game Rental","Sports Bars","Fast Food","Pretzels","Dive Bars","Grocery","Reflexology","Chicken Wings","Kitchen & Bath","Outlet Stores","Plus Size Fashion","Yoga","Boxing","Gas Stations","Security Systems","Dog Walkers","Banks & Credit Unions","Home Cleaning","Gelato","Steakhouses","Sushi Bars","Head Shops","Custom Cakes","Sports Clubs","Pediatric Dentists","Musical Instruments & Teachers","Towing","Home Theatre Installation","Toy Stores","Medical Centers","Preschools","Gymnastics","Lingerie","Cupcakes","Professional Services","Libraries","Dentists","Cocktail Bars","Taiwanese","Bespoke Clothing","Candle Stores","Books","Automotive","Nightlife","Furniture Stores","Arts & Crafts","Endodontists","Auto Repair","Tanning","Rugs","Diners","Jewelry Repair","Printing Services","Japanese","Convenience Stores","Fish & Chips","Flooring","Estate Planning Law","Pharmacy","Session Photography","Travel Services","Massage","Breakfast & Brunch","Home & Garden","Insurance","Holiday Decorations","Security Services","American (New)","Tennis","Eyewear & Opticians","Health Markets","Home Services","Electronics Repair","Food Delivery Services","Oral Surgeons","Gutter Services","Pumpkin Patches","Auto Loan Providers","Jewelry","Vintage & Consignment","Real Estate","Gyms","Men's Hair Salons","Framing","Auto Parts & Supplies","Counseling & Mental Health","Juice Bars & Smoothies","Fashion","Bridal","Generator Installation/Repair","American (Traditional)","Trainers","Taxis","Traditional Chinese Medicine","Summer Camps","Cheese Shops","Laboratory Testing","Shopping Centers","Property Management","Veterinarians","Formal Wear","Pediatricians","Local Services","Internet Service Providers","Personal Injury Law","Hotels & Travel","Baby Gear & Furniture","Party & Event Planning","Salvadoran","Landmarks & Historical Buildings","Urgent Care","Ramen","Financial Services","Signmaking","Butcher","Home Health Care","Pets","Women's Clothing","Optometrists","Noodles","Cosmetic Dentists","Personal Shopping","Barbeque","Used Car Dealers","Shopping","Art Classes","Beer","Florists","Latin American","Pool & Hot Tub Service","Hospitals","Oil Change Stations","Cosmetology Schools","Pizza","Art Galleries","Water Heater Installation/Repair","Tanning Beds","Hospice","Gastropubs","Buffets","Makeup Artists","Psychiatrists","Farms","Education","Pet Sitting","Seafood","Videographers","Shipping Centers","Rehabilitation Center","Candy Stores","Commercial Truck Repair","Drugstores","Pet Stores","Kids Activities","Beer Bar","Mexican","Waffles","Greek","Waxing","Mobile Phones","Arts & Entertainment","Specialty Food","Telecommunications","Transmission Repair","Reiki","Meat Shops","Food","Accessories","Pasta Shops","Laser Hair Removal","Gift Shops","Cosmetics & Beauty Supply","Festivals","Thai","Beauty & Spas","Vegetarian","Hobby Shops","Transportation","Vape Shops","Career Counseling","Roadside Assistance","Martial Arts","Child Care & Day Care","Appliances","Delis","Wine Bars","Hotels","Department Stores","Carpeting","Retirement Homes","Day Spas","Hair Extensions","Lawyers","Accountants","Attraction Farms","Indian","IT Services & Computer Repair","Ophthalmologists","Bowling","Dry Cleaning & Laundry","Laundry Services","Active Life","Korean","Roofing","Children's Clothing","Asian Fusion","Post Offices","Teeth Whitening","Interior Design","Pet Groomers","Electronics","Acupuncture","Emergency Rooms","Lounges","Tex-Mex","Bakeries","Tui Na","Sewing & Alterations","Wine & Spirits","Pet Training","Wholesale Stores","Sports Medicine","Doctors","Local Flavor","Chicken Shop","Ice Cream & Frozen Yogurt","Orthodontists","Lighting Fixtures & Equipment","Art Supplies","Mobile Phone Repair","Salad","Pet Boarding","Health & Medical","Hot Dogs","Mattresses","Cinema","Dermatologists","Nurseries & Gardening","Nutritionists","Prenatal/Perinatal Care","Landscaping","Eyebrow Services","Tree Services","Party Supplies","Masonry/Concrete","Notaries","Public Services & Government","Used","Battery Stores","Gluten-Free","Home Automation","Commercial Truck Dealers","Spray Tanning","Hot Tub & Pool","Appliances & Repair","Venues & Event Spaces","Hair Removal","Car Rental","Auto Customization","Auto Glass Services","Painters","Vitamins & Supplements","Wheel & Rim Repair","Home Decor","Medical Spas","Self Storage","Car Wash","Keys & Locksmiths"]])
    #similarity_matrix = cosine_similarity(trainset[top_categories])

    community_recommendations = {}

    for user_id, group in trainset.groupby('user_id'):
        #user_ratings = set(group[group['rating'] >= 3]['business_id'])
        user_ratings = set(group['business_id'])
        user_recommendations = {}   
        
        for idx, (business_id, _) in enumerate(group[['business_id', 'rating']].values):
            similar_indices = np.argsort(similarity_matrix[idx])[::-1][1:]
            similar_businesses = [(trainset.iloc[sim_index]['business_id'], similarity_matrix[idx][sim_index]) for sim_index in similar_indices]
            
            for sim_business, sim_score in similar_businesses:
                if sim_business not in user_ratings: 
                    if sim_business not in user_recommendations:
                        user_recommendations[sim_business] = sim_score
                    else:
                        user_recommendations[sim_business] += sim_score
        
        user_top_recommendations = sorted(user_recommendations.keys(), key=lambda x: user_recommendations[x], reverse=True)[:top_n]
        community_recommendations[user_id] = user_top_recommendations
        
    return community_recommendations

In [ ]:
content_rec_per_community = {}

for community_id, community in communities.items():
    if community_id in community_trainsets_content:
        recommendation = content_based_recommendations(community_trainsets_content[community_id])
        content_rec_per_community[community_id] = recommendation

#### Evaluation of content-based

In [1013]:
def calculate_precision(community_test_dfs, recommendations_per_community, communities):
    all_hits = 0
    all_total = 0
    all_precisions = []
    all_recalls = []
    all_f1 = []

    for community_id, community in communities.items():
        if community_id in community_trainsets_content:
            test_df = community_test_dfs[community_id]
            hits = 0
            total = 0
            total_gt = 0
            for user_id, recommendations in recommendations_per_community[community_id].items():
                gt = test_df[(test_df['user_id'] == user_id) & (test_df['rating'] >= 3)].item_id.to_list()
                if len(recommendations) > 0 and len(gt) > 0:
                    hits += len(set(gt).intersection(set(recommendations)))
                    total += len(recommendations)
                    total_gt += len(gt)
            all_hits += hits
            all_total += total
            if total != 0:
                precision = hits / total
                recall = hits / total_gt
                if precision + recall == 0:
                    f1 = 0
                else:
                    f1 = 2 * (precision * recall) / (precision + recall)
                all_precisions.append(precision)
                all_recalls.append(recall)
                all_f1.append(f1)

    overall_precision = np.mean(all_precisions) if all_precisions else 0
    overall_recall = np.mean(all_recalls) if all_recalls else 0
    overall_f1 = np.mean(all_f1) if all_f1 else 0

    overall_hits_total = f"{all_hits} / {all_total}" if all_total != 0 else "No test data"

    return overall_hits_total, overall_precision, overall_recall, overall_f1


In [1014]:
community_hits, precisions, recall, f1 = calculate_precision(community_test_dfs, content_rec_per_community, communities)
print("Community Hits:")
print(community_hits)
print("Precision:")
print(precisions)
print("Recall:")
print(recall)
print("F1:")
print(f1)

Community Hits:
19 / 1104
Precision:
0.030466631096883197
Recall:
0.07279509038598157
F1:
0.04241204151859029


### Hybrid rec system (user based with content based)

In [1015]:
'''
Join the user-based and content-based recommendations
'''
def join_recommendations(user_based, content_based, num_of_recommendations=3):
    hybrid_recommendations = {}

    for community_id, community_recommendations in content_based.items():
        hybrid_recommendations[community_id] = {}
        for user_id, user_based_rec in user_based.items():
            content_rec = community_recommendations.get(user_id, [])  # Get content-based recommendations if available
            combined_rec = []

            common_businesses = set([rec[0] for rec in user_based_rec]).intersection(set(content_rec))
            for rec in user_based_rec:
                if rec[0] in common_businesses:
                    combined_rec.append(rec[0])

            for rec in user_based_rec:
                if rec[0] not in common_businesses and len(combined_rec) < num_of_recommendations:
                    combined_rec.append(rec[0])

            for rec in content_rec:
                if rec not in common_businesses and len(combined_rec) < num_of_recommendations:
                    combined_rec.append(rec)


            hybrid_recommendations[community_id][user_id] = combined_rec[:num_of_recommendations]

    return hybrid_recommendations


In [1016]:

hybrid_recommendations = join_recommendations(user_recommendations, content_rec_per_community)

community_hits, precisions, recall, f1 = calculate_precision(community_test_dfs, hybrid_recommendations, communities)
print("Community Hits:")
print(community_hits)
print("Precision:")
print(precisions)
print("Recall:")
print(recall)
print("F1:")
print(f1)

Community Hits:
518 / 1141
Precision:
0.6980484713897848
Recall:
0.9956178410274211
F1:
0.7791715983577455


### Recommend the most popular restaurants

In [1017]:
pop_recommendations = {}

for community_id, trainset in community_trainsets.items():
    popular_businesses = popular_recommendations(trainset, 3)
    pop_recommendations[community_id] = {}  
    for user_id in trainset.ur:
        pop_recommendations[community_id][trainset.to_raw_uid(user_id)] = popular_businesses


community_hits, precisions, recall, f1 = calculate_precision(community_test_dfs, pop_recommendations, communities)
print("Community Hits:")
print(community_hits)
print("Precision:")
print(precisions)
print("Recall:")
print(recall)
print("F1:")
print(f1)
    

Community Hits:
49 / 1175
Precision:
0.01735214211531974
Recall:
0.047409686394150635
F1:
0.025310094831552596
